# PySpark and Big Data Project -  Solutions

## Task Group 1 - Analyzing Common Crawl Data with RDDs

### Task 1

Initialize a new Spark Context and read in the domain graph as an RDD.

In [1]:
# Import required modules
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

# Get SparkContext
sc = spark.sparkContext

In [2]:
# Read Domains CSV File into an RDD
common_crawl_domain_counts = sc.textFile('./crawl/cc-main-limited-domains.csv')

# Display first few domains from the RDD
common_crawl_domain_counts.take(10)

['367855\t172-in-addr\tarpa\t1',
 '367856\taddr\tarpa\t1',
 '367857\tamphic\tarpa\t1',
 '367858\tbeta\tarpa\t1',
 '367859\tcallic\tarpa\t1',
 '367860\tch\tarpa\t1',
 '367861\td\tarpa\t1',
 '367862\thome\tarpa\t7',
 '367863\tiana\tarpa\t1',
 '367907\tlocal\tarpa\t1']

### Task 2

Apply `fmt_domain_graph_entry` over `common_crawl_domain_counts` and save the result as a new RDD named `formatted_host_counts`.

In [3]:
def fmt_domain_graph_entry(entry):
    """
    Formats a Common Crawl domain graph entry. Extracts the site_id, 
    top-level domain (tld), domain name, and subdomain count as seperate items.
    """

    # Split the entry on delimiter ('\t') into site_id, domain, tld, and num_subdomains
    site_id, domain, tld, num_subdomains = entry.split('\t')        
    return int(site_id), domain, tld, int(num_subdomains)

In [4]:
# Apply `fmt_domain_graph_entry` to the raw data RDD with the `.map()` method
formatted_host_counts = common_crawl_domain_counts\
    .map(lambda e: fmt_domain_graph_entry(e))

# Display the first few entries of the new RDD
formatted_host_counts.take(10)

[(367855, '172-in-addr', 'arpa', 1),
 (367856, 'addr', 'arpa', 1),
 (367857, 'amphic', 'arpa', 1),
 (367858, 'beta', 'arpa', 1),
 (367859, 'callic', 'arpa', 1),
 (367860, 'ch', 'arpa', 1),
 (367861, 'd', 'arpa', 1),
 (367862, 'home', 'arpa', 7),
 (367863, 'iana', 'arpa', 1),
 (367907, 'local', 'arpa', 1)]

### Task 3

Apply `extract_subdomain_counts` over `common_crawl_domain_counts` and save the result as a new RDD named `host_counts`.

In [5]:
def extract_subdomain_counts(entry):
    """
    Extract the subdomain count from a Common Crawl domain graph entry.
    """
    
    # Split the entry on delimiter ('\t') into site_id, domain, tld, and num_subdomains
    site_id, domain, tld, num_subdomains = entry.split('\t')
    
    # return ONLY the num_subdomains
    return int(num_subdomains)


# Apply `extract_subdomain_counts` to the raw data RDD
host_counts = common_crawl_domain_counts\
    .map(lambda e: extract_subdomain_counts(e))

# Display the first few entries
host_counts.take(10)

[1, 1, 1, 1, 1, 1, 1, 7, 1, 1]

### Task 4

Using `host_counts`, calculate the total number of subdomains across all domains in the dataset, save the result to a variable named `total_host_counts`.

In [6]:
# Reduce the RDD to a single value, the sum of subdomains, with a lambda function
# as the reduce function
total_host_counts = host_counts\
    .reduce(lambda a, b: a + b)

# Display result count
total_host_counts

595466

In [7]:
# Alternative Solution - Import `add` from Python's `operator` package
from operator import add

# Reduce the RDD to a single value, the sum of subdomains, with `add`
# as the reduce function
total_host_counts = host_counts\
    .reduce(add)

# Display result count
total_host_counts

595466

### Task 5

Stop the current `SparkSession` and `SparkContext` before moving on to analyze the data with SparkSQL

In [8]:
# Stop the SparkContext and the SparkSession with spark.stop()
spark.stop()

## Task Group 2 - Exploring Domain Counts with PySpark DataFrames and SQL

### Task 6

Create a new `SparkSession` and assign it to a variable named `spark`.

In [9]:
from pyspark.sql import SparkSession

# Create a new SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

### Task 7

Read `./crawl/cc-main-limited-domains.csv` into a new Spark DataFrame named `common_crawl`.

In [10]:
# Read the target file (`./crawl/cc-main-limited-domains.csv`) into a DataFrame (`common_crawl`)

common_crawl = spark.read \
    .option('delimiter', '\t') \
    .option('inferSchema', True) \
    .csv('./crawl/cc-main-limited-domains.csv')

# Display the DataFrame to the notebook
common_crawl.show(5, truncate=False)

+------+-----------+----+---+
|_c0   |_c1        |_c2 |_c3|
+------+-----------+----+---+
|367855|172-in-addr|arpa|1  |
|367856|addr       |arpa|1  |
|367857|amphic     |arpa|1  |
|367858|beta       |arpa|1  |
|367859|callic     |arpa|1  |
+------+-----------+----+---+
only showing top 5 rows



### Task 8

Rename the DataFrame's columns to the following: 

- site_id
- domain
- top_level_domain
- num_subdomains


In [11]:
# Rename the DataFrame's columns with `withColumnRenamed()`
common_crawl = common_crawl\
    .withColumnRenamed("_c0", "site_id")\
    .withColumnRenamed("_c1", "domain")\
    .withColumnRenamed("_c2", "top_level_domain")\
    .withColumnRenamed("_c3", "num_subdomains")
  
# Display the first few rows of the DataFrame and the new schema in the notebook
common_crawl.show(5, truncate=False)
common_crawl.printSchema()

+-------+-----------+----------------+--------------+
|site_id|domain     |top_level_domain|num_subdomains|
+-------+-----------+----------------+--------------+
|367855 |172-in-addr|arpa            |1             |
|367856 |addr       |arpa            |1             |
|367857 |amphic     |arpa            |1             |
|367858 |beta       |arpa            |1             |
|367859 |callic     |arpa            |1             |
+-------+-----------+----------------+--------------+
only showing top 5 rows

root
 |-- site_id: integer (nullable = true)
 |-- domain: string (nullable = true)
 |-- top_level_domain: string (nullable = true)
 |-- num_subdomains: integer (nullable = true)



In [12]:
# Alternative Solution
#
# Just like RDDs, DataFrames also have a `.toDF()` method that returns a new DataFrame with 
# specified column names. This can be helpful when you have many columns to rename.

common_crawl.toDF("site_id", "domain", "top_level_domain", "num_subdomains")\
    .show(5, truncate=False)

+-------+-----------+----------------+--------------+
|site_id|domain     |top_level_domain|num_subdomains|
+-------+-----------+----------------+--------------+
|367855 |172-in-addr|arpa            |1             |
|367856 |addr       |arpa            |1             |
|367857 |amphic     |arpa            |1             |
|367858 |beta       |arpa            |1             |
|367859 |callic     |arpa            |1             |
+-------+-----------+----------------+--------------+
only showing top 5 rows



## Task Group 3 - Reading and Writing Datasets to Disk

### Task 9

Save the `common_crawl` DataFrame as parquet files in a directory called `./results/common_crawl/`.

In [13]:
# Save the `common_crawl` DataFrame to a series of parquet files in `./results/common_crawl/` 
# with `DataFrame.write.parquet()`

common_crawl\
    .write\
    .parquet('./results/common_crawl/', mode="overwrite")

### Task 10

Read `./results/common_crawl/` into a new DataFrame to confirm our DataFrame was saved properly.

In [14]:
# Read from parquet directory
common_crawl_domains = spark.read\
    .parquet('./results/common_crawl/')

# Display the first few rows of the DataFrame and the schema in the notebook
common_crawl_domains.show(5, truncate=False)
common_crawl_domains.printSchema()

+-------+-----------+----------------+--------------+
|site_id|domain     |top_level_domain|num_subdomains|
+-------+-----------+----------------+--------------+
|367855 |172-in-addr|arpa            |1             |
|367856 |addr       |arpa            |1             |
|367857 |amphic     |arpa            |1             |
|367858 |beta       |arpa            |1             |
|367859 |callic     |arpa            |1             |
+-------+-----------+----------------+--------------+
only showing top 5 rows

root
 |-- site_id: integer (nullable = true)
 |-- domain: string (nullable = true)
 |-- top_level_domain: string (nullable = true)
 |-- num_subdomains: integer (nullable = true)



## Task Group 4 - Querying Domain Counts with PySpark DataFrames and SQL

### Task 11

Create a local temporary view from `common_crawl_domains`

In [15]:
# Create a temporary view in the metadata for this `SparkSession` to make the data
# queryable with `sparkSession.sql()`
common_crawl_domains.createOrReplaceTempView("crawl")

### Task 12

Calculate the total number of domains for each top-level domain in the dataset.

In [16]:
# Aggregate the DataFrame using DataFrame methods
#
# .groupby('top_level_domain')       -> Aggregate on top_level_domain
# .count()                           -> The aggregate function to apply 
# .orderBy('count', ascending=False) -> Order from highest to lowest number of domains

common_crawl_domains\
    .groupby('top_level_domain')\
    .count()\
    .orderBy('count', ascending=False)\
    .show(10, truncate=False)

+----------------+-----+
|top_level_domain|count|
+----------------+-----+
|edu             |18547|
|gov             |15007|
|travel          |6313 |
|coop            |5319 |
|jobs            |3893 |
|post            |117  |
|map             |34   |
|arpa            |11   |
+----------------+-----+



In [17]:
# Aggregate the DataFrame using SQL's `COUNT`, `GROUP BY`, and `ORDER BY`
spark.sql(
    """
    SELECT 
        top_level_domain, 
        COUNT(domain) AS count
    FROM crawl
    GROUP BY top_level_domain
    ORDER BY COUNT(domain) DESC
    """
).show(10, truncate=False)

+----------------+-----+
|top_level_domain|count|
+----------------+-----+
|edu             |18547|
|gov             |15007|
|travel          |6313 |
|coop            |5319 |
|jobs            |3893 |
|post            |117  |
|map             |34   |
|arpa            |11   |
+----------------+-----+



### Task 13

Calculate the total number of subdomains for each top-level domain in the dataset.

In [18]:
# Aggregate the DataFrame using DataFrame methods
#
# .groupby('top_level_domain')                     -> Aggregate on top_level_domain
# .sum('num_subdomains')                           -> The aggregate function to apply 
# .orderBy('sum(num_subdomains)', ascending=False) -> Order from highest to lowest total subdomains

common_crawl_domains\
    .groupby('top_level_domain')\
    .sum('num_subdomains')\
    .orderBy('sum(num_subdomains)', ascending=False)\
    .show(10, truncate=False)

+----------------+-------------------+
|top_level_domain|sum(num_subdomains)|
+----------------+-------------------+
|edu             |484438             |
|gov             |85354              |
|travel          |10768              |
|coop            |8683               |
|jobs            |6023               |
|post            |143                |
|map             |40                 |
|arpa            |17                 |
+----------------+-------------------+



In [19]:
# Aggregate the DataFrame using SQL's `COUNT`, `GROUP BY`, and `ORDER BY`
spark.sql(
    """
    SELECT 
        top_level_domain, 
        SUM(num_subdomains) AS total_count
    FROM crawl
    GROUP BY top_level_domain
    ORDER BY SUM(num_subdomains) DESC
    """
).show(10, truncate=False)

+----------------+-----------+
|top_level_domain|total_count|
+----------------+-----------+
|edu             |484438     |
|gov             |85354      |
|travel          |10768      |
|coop            |8683       |
|jobs            |6023       |
|post            |143        |
|map             |40         |
|arpa            |17         |
+----------------+-----------+



### Task 14

How many sub-domains does `nps.gov` have? Filter the dataset to that website's entry, display the columns `top_level_domain`, `domain`, and `num_subdomains` in your result.

In [20]:
# Filter the DataFrame using DataFrame Methods `select` and `filter`

common_crawl_domains\
    .select(['top_level_domain', 'domain', 'num_subdomains'])\
    .filter(common_crawl_domains.domain == "nps")\
    .filter(common_crawl_domains.top_level_domain == "gov")\
    .show(10, truncate=False)

+----------------+------+--------------+
|top_level_domain|domain|num_subdomains|
+----------------+------+--------------+
|gov             |nps   |178           |
+----------------+------+--------------+



In [21]:
# Filter the DataFrame using a SQL `WHERE` statement

spark.sql(
    """
    SELECT top_level_domain, domain, num_subdomains
    FROM crawl
    WHERE domain = "nps" 
    AND top_level_domain = 'gov'
    """
).show(truncate=False)

+----------------+------+--------------+
|top_level_domain|domain|num_subdomains|
+----------------+------+--------------+
|gov             |nps   |178           |
+----------------+------+--------------+



### Task 15

Close the `SparkSession` and underlying `SparkContext`.

In [22]:
# Stop the notebook's `SparkSession` and `SparkContext`
spark.stop()